# Azure Open AI - Demo 4 Speech summarizing

**Before getting started!**
- Create an Azure Speech service and create GitHub Secrets for it (or use and .env file uncommenting some sections)
    - AZURE_SPEECH_KEY and AZURE_SPEECH_REGION
- Make sure speech module **azure-cognitiveservices-speech** is installed (already executed automatically in the GH Codespace from requirements.txt)

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import openai
import os
import sys
import time

# from dotenv import load_dotenv
from IPython.display import Audio

In [ ]:
sys.version

In [ ]:
print("openai version =", openai.__version__)

In [ ]:
# load_dotenv("azure.env")

openai.api_type: str = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
# openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_version = os.getenv("AZURE_OPENAI_MODEL_CHAT_VERSION")

openai.engine = os.getenv("AZURE_OPENAI_MODEL_CHAT")

azure_speech_key = os.getenv("AZURE_SPEECH_KEY")
azure_speech_region = os.getenv("AZURE_SPEECH_REGION")

## 1. Convert Audio To Text Using Azure Speech To Text  

Recognize feature provided by Azure Speech to convert Audio to Text

In [ ]:
def recognize_speech_from_file(filename):
    """This function transcribes an audio file into its corresponding textual output."""

    # setup global parameters
    global done
    done = False
    global recognized_text_list
    recognized_text_list = []

    # initiate the Speech Service
    speech_config = speechsdk.SpeechConfig(
        subscription=azure_speech_key, region=azure_speech_region
    )
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    # create a Speech Recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config, audio_config=audio_config
    )

    # define a call-back stop function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print("CLOSING on {}".format(evt))
        global done
        done = True

    # define a call-back recognize function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        print("RECOGNIZED: {}".format(evt.result.text))

    # connect callbacks to the recognizer events
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(
        lambda evt: print("STT SESSION STARTED: {}".format(evt))
    )
    speech_recognizer.session_stopped.connect(
        lambda evt: print("STT SESSION STOPPED {}".format(evt))
    )

    # stop continuous speech recognition
    speech_recognizer.session_stopped.connect(stop_cb)

    # start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(0.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

## 2. Summarize Audio - Positive Example 

In [ ]:
# play the audio sound
Audio("documents/good_review.wav", autoplay=False)

In [ ]:
text = recognize_speech_from_file("documents/good_review.wav")

In our case we will use the GPT3 model used in other demos

In [ ]:
#model = "text-davinci-003"
print("openai.engine =", openai.engine)

Analyze the provided Audio

In [ ]:
# create a prompt for text analytics
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """
# results = openai.Completion.create(
#     engine=model, prompt=prompt, temperature=0, max_tokens=1000
# )
results = openai.Completion.create(
     engine=openai.engine, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])

## 3. Summarize Audio - Negative Example 

In [ ]:
Audio("documents/bad_review.wav", autoplay=False)

In [ ]:
text = recognize_speech_from_file("documents/bad_review.wav")

In [ ]:
prompt = f"""Text:

             ###
             {" ".join(text)}
             ###
             
             Tasks:
             1. Summarize the above text in a short and concise manner.
             2. Determine the above text as whether Positive or Negative.
             3. Classify the above text into a topic.
             4. Whether the above text is Positive or Negative, 
                suggest 3 ways to handle it.
                
             Answer the above 4 tasks, one at a time. 
          """

# openai results
results = openai.Completion.create(
    engine=openai.engine, prompt=prompt, temperature=0, max_tokens=1000
)

print(results["choices"][0]["text"])